### **Initialization**

In [105]:
stop_words  = open("stopwords.txt", "r")

In [106]:
stop_words_list = []
for line in stop_words:
    line = line.strip('\n').split()
    stop_words_list.append(line[0])

#### Statements to be imported

In [40]:
import inltk
from inltk.inltk import tokenize, get_embedding_vectors, predict_next_words, get_sentence_encoding, get_sentence_similarity, get_similar_sentences

In [ ]:
import pyiwn
iwn = pyiwn.IndoWordNet()

### **Input**

In [1]:
input_sentence = input("Enter input sentence:")

Enter input sentence: मुझे खाना बहुत पसंद है


### **Preprocessing Data**

In [5]:
def preprocess_sentence(sentence):
    tokenized_ips = tokenize(sentence, 'hi')
    for i in range(len(tokenized_ips)):
        tokenized_ips[i] = tokenized_ips[i][1:]
    preprocessed_ips = ""
    for token in tokenized_ips:
        if token not in stop_words_list:
            preprocessed_ips += (" "+token)
    return preprocessed_ips

In [6]:
preprocess_sentence(input_sentence)

' मुझे खाना पसंद'

### **Ambiguous Word Input**

In [7]:
amb_word = input("Enter ambiguous word:")

Enter ambiguous word: खाना


In [8]:
import pandas as pd
from googletrans import Translator

import nltk
import string
import re

from nltk.stem import WordNetLemmatizer

from englisttohindi.englisttohindi import EngtoHindi

In [115]:
def HindiLemmetizer(word):
    temp=translator.translate(word).text
    updated_temp=lemmatizer.lemmatize(temp)
    return (EngtoHindi(updated_temp)).convert

### **pyiwn**

#### Data Gathering

In [267]:
import operator

def generate_synonyms_set(root_word, input_sentence, lst):
    # amb_word = HindiLemmetizer(amb_word)
    synsets = iwn.synsets(root_word)
    syn_head_sim = {}
    tst = []
    
    for synset in synsets:
        head = synset.head_word()
        ratio = get_sentence_similarity(synset.examples()[0], input_sentence ,'hi')
        print(head, ":", synset.examples()[0], ratio)
        
        if (head not in syn_head_sim):
            syn_head_sim[head] = ratio
        elif (head in syn_head_sim) and syn_head_sim[head] < ratio:
            syn_head_sim[head] = ratio
            
        if head == root_word and len(synset.lemma_names()) > 1:
            tst.append((synset.lemma_names()[1], ratio))
        
    print("Before:", syn_head_sim)
    syn_head_sim = dict(sorted(syn_head_sim.items(), key=operator.itemgetter(1),reverse=True))
    synonyms_list = []
    
    print("Printing dict:", syn_head_sim)
    for x in list(syn_head_sim)[0:7]:
        if x == root_word:
            for i, j in tst:
                print(i, j)
                print(syn_head_sim[x])
                if j == syn_head_sim[x]:
                    print("True")
                    lst.append(i)
        synonyms_list.append(x)
    
    print(synonyms_list)
    print("lst:", lst)
    return synonyms_list

### **Cosine Similarity**

In [58]:
def generate_sense_replaced_sentences(synonyms_set, input_sentence, amb_word):
    sentences_dict = {}
    for syn in synonyms_set:
        dummy = input_sentence
        sentences_dict[syn] = dummy.replace(amb_word, syn)
    return sentences_dict

In [210]:
def generate_highly_similar_sentences(sentences_dict):
    high_sim_sent = {}
    for sense, sent in sentences_dict.items():
        sim = get_sentence_similarity(sent, input_sentence, 'hi')
        if sim >= 0.9:
            print(">0.9->", sim, sent)
            high_sim_sent[sense] = sent
    return high_sim_sent

In [50]:
print(high_sim_sent)
print(input_sentence)
for sent in high_sim_sent:
    print(sent, get_sentence_similarity(sent, input_sentence, 'hi'))

['मुझे आहार बहुत पसंद है', 'मुझे बर्बाद करना बहुत पसंद है', 'मुझे भोजन बहुत पसंद है']
मुझे खाना बहुत पसंद है


मुझे आहार बहुत पसंद है 0.9283260703086853


मुझे बर्बाद करना बहुत पसंद है 0.9006214141845703


मुझे भोजन बहुत पसंद है 0.9286918640136719


In [18]:
ppc_high_sim_sent = []
for sent in high_sim_sent:
    ppc_high_sim_sent.append(preprocess_sentence(sent))

In [19]:
ppc_high_sim_sent

[' मुझे आहार पसंद', ' मुझे बर्बाद पसंद', ' मुझे भोजन पसंद']

### **MuRIL**

#### Model

In [20]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

In [21]:
def get_model(model_url, max_seq_length):
    inputs = dict(
        input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
        input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
        input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    )

    muril_layer = hub.KerasLayer(model_url, trainable=True)
    outputs = muril_layer(inputs)

    assert 'sequence_output' in outputs
    assert 'pooled_output' in outputs
    assert 'encoder_outputs' in outputs
    assert 'default' in outputs
    return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer

In [22]:
def create_input(input_strings, tokenizer, max_seq_length):
    input_ids_all, input_mask_all, input_type_ids_all = [], [], []
    for input_string in input_strings:
        input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        sequence_length = min(len(input_ids), max_seq_length)
    
        if len(input_ids) >= max_seq_length:
            input_ids = input_ids[:max_seq_length]
        else:
            input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

        input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

        input_ids_all.append(input_ids)
        input_mask_all.append(input_mask)
        input_type_ids_all.append([0] * max_seq_length)
  
    return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [23]:
def encode(input_text):
    input_ids, input_mask, input_type_ids = create_input(input_text, 
                                                       tokenizer, 
                                                       max_seq_length)
    inputs = dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
    )
    return muril_model(inputs)

In [24]:
max_seq_length = 128
muril_model, muril_layer = get_model(model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=max_seq_length)
     

vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

2023-05-16:10:12:31,839 INFO     [resolver.py:109] Using C:\Users\thakr\AppData\Local\Temp\tfhub_modules to cache modules.


#### Returning best Euclidean distance sentence

In [53]:
def tokenize_sentences(ppc_high_sim_sent):
    tokenized_sentences = []
    for sentence in ppc_high_sim_sent:
        s = sentence.strip()
        tokenized_sentences.append(s.split(" "))
    return tokenized_sentences

In [52]:
sent_without_mujhe = sentences

In [205]:
def generate_muril_output(sent):
    embeddings = encode(sent)
    tot_dist = 0
    print("->", sent)
    for index in range(len(sent)):
        tot_dist += distance.euclidean(np.array(embeddings[index]), np.array(embeddings[(index+1)%len(sent)]))
        print(sent[index], "\t", sent[(index+1)%len(sent)], "\t:", distance.euclidean(np.array(embeddings[index]), np.array(embeddings[(index+1)%len(sent)])))
    print("avg : ", tot_dist/len(sent), end = "\n\n")
    return tot_dist/len(sent)

In [37]:
for sentence in sentences:
    print("->", sentence[1])
    generate_muril_output(sentence)

-> पसंद
आहार 	 पसंद 	: 0.011560117825865746
पसंद 	 आहार 	: 0.011560117825865746
avg :  0.011560117825865746

-> पसंद
बर्बाद 	 पसंद 	: 0.008085211738944054
पसंद 	 बर्बाद 	: 0.008085211738944054
avg :  0.008085211738944054

-> पसंद
भोजन 	 पसंद 	: 0.010638106614351273
पसंद 	 भोजन 	: 0.010638106614351273
avg :  0.010638106614351273



In [275]:
def wsd_model(input_str, amb_word, root_word):    
    lst = []
    synonyms_set = generate_synonyms_set(root_word, input_str, lst)
    
    sentences_dict = generate_sense_replaced_sentences(synonyms_set, input_sentence, amb_word)
    # print(sentences_dict)
    
    high_sim_sent = generate_highly_similar_sentences(sentences_dict)
    # print(high_sim_sent)
    senses = []
    for sense in high_sim_sent.keys():
        senses.append(sense)
    
    ppc_high_sim_sent = []
    for sent in high_sim_sent.values():
        ppc_high_sim_sent.append(preprocess_sentence(sent))
        
    tokenized_sentences = tokenize_sentences(ppc_high_sim_sent)
    
    euclidean_dist = {}
    for index in range(len(tokenized_sentences)):
        euclidean_dist[senses[index]] = generate_muril_output(tokenized_sentences[index])
        
    if bool(euclidean_dist) == False:
        return input_str
    best_sense = list(euclidean_dist.keys())[0]
    min_dist = list(euclidean_dist.values())[0]
    min_dist
    for sense, dist in euclidean_dist.items():
        if min_dist > dist:
            min_dist = dist
            best_sense = sense
            
    if best_sense == root_word and len(lst):
        best_sense = lst[0]
    
    input_str = input_str.replace(amb_word, best_sense)
    return input_str

In [273]:
input_sentence = input("Enter input sentence:")

Enter input sentence: हम ताश खेल रहे हैं


In [270]:
amb_word = input("Enter ambiguous word:")

Enter ambiguous word: ताश


In [271]:
root_word = input("Enter root word:")

Enter root word: ताश


In [276]:
wsd_model(input_sentence, amb_word, root_word)

ज़री : सीता ने शादी में ज़री की साड़ी पहनी । 0.12953366339206696


ताश : उसने गुस्से में ताश को फाड़ दिया । 0.12842033803462982


ताश : कुछ लोग बाग में ताश खेल रहे हैं । 0.3216466009616852
Before: {'ज़री': 0.12953366339206696, 'ताश': 0.3216466009616852}
Printing dict: {'ताश': 0.3216466009616852, 'ज़री': 0.12953366339206696}
पत्ता 0.12842033803462982
0.3216466009616852
['ताश', 'ज़री']
lst: []


>0.9-> 1.0 हम ताश खेल रहे हैं


-> ['ताश', 'खेल']
ताश 	 खेल 	: 0.01976960524916649
खेल 	 ताश 	: 0.01976960524916649
avg :  0.01976960524916649



'हम ताश खेल रहे हैं'

In [229]:
generate_synonyms_set("फल", input_sentence)

फल 662 ['उसने फल की दुकान से एक किलो आम खरीदा ।'] 0.16055671870708466


नतीजा 2035 ['उसके काम का नतीजा बहुत ही बुरा निकला ।'] 0.20405006408691406


जायफल 2931 ['बच्चों को खाँसी आने पर जायफल को घिसकर पिलाया जाता है ।'] 0.09860213100910187


फल 4314 ['इस तीर का फल बहुत नुकीला है ।'] 0.38117098808288574


ढाल 5175 ['ढाल योद्धाओं को सुरक्षा प्रदान करती है ।'] 0.11565416306257248


ब्याज 7350 ['श्याम ब्याज पर पैसा देता है ।'] 0.15408121049404144


बोर्ड 10826 ['गुरुजी बोर्ड पर गणित समझा रहे हैं ।', 'सड़क के किनारे जगह-जगह पर दिग्दर्शक फलक लगे हुए हैं ।'] 0.00014742097118869424


राशिफल 38346 ['बहुत लोग राशिफल पर विश्वास नहीं करते हैं ।'] 0.05805698409676552


फल 38347 ['क्षेत्रफल, गुणनफल, योगफल, भागफल आदि फल हैं ।'] 0.023139819502830505


['नतीजा', 'ब्याज', 'ढाल', 'जायफल', 'राशिफल', 'फल', 'बोर्ड']

In [192]:
generate_synonyms_set("योग", input_sentence)

['उपाय', 'सिद्धांत', 'ध्यानयोग', 'नतीजा', 'अवसर', 'विश्वासघाती', 'मुहूर्त']


['उपाय',
 'सिद्धांत',
 'ध्यानयोग',
 'नतीजा',
 'अवसर',
 'विश्वासघाती',
 'मुहूर्त',
 'योगशास्त्र',
 'संबंध',
 'नाम',
 'नैयायिक',
 'योग',
 'सवारी',
 'उपयुक्ततता',
 'चतुराई',
 'ईश्वर',
 'प्रयोग',
 'लाभ',
 'योग छंद',
 'छल',
 'योगदान',
 'तपस्या',
 'औषध',
 'संपत्ति',
 'जोड़',
 'वशीकरण',
 'आय',
 'व्यवसाय']

In [165]:
get_sentence_similarity("वह प्रतिदिन योग करता है ।", input_sentence, 'hi')

0.2700381577014923

In [166]:
get_sentence_similarity("इन अंकों का जोड़ सावधानीपूर्वक करना", input_sentence, 'hi')

0.12754514813423157

In [278]:
import gradio as gr

In [281]:
outputs = gr.outputs.Textbox()
app = gr.Interface(fn=wsd_model, inputs=['text','text','text'], outputs=outputs,description="Hindi Word Sensse Disambiguation")

In [282]:
app.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


ईश्वर : ईश्वर सर्वव्यापी है । 0.34982702136039734


नाम : हमारे प्राचार्य जी का नाम श्री पुष्पक भट्टाचार्य है । 0.339528888463974


जोड़ : इन अंकों का जोड़ सावधानीपूर्वक करना । 0.25820520520210266


जोड़ : इन संख्याओं का जोड़ बीस आया । 0.21522517502307892


...

शांति : अंधेरी रात में निस्तब्धता छाई हुई थी । 0.07162833958864212


शान्ति : शांति का विवाह अथर्व ऋषि के साथ हुआ था । 0.10455728322267532


शांति : युद्ध के बाद देश में शांति है । 0.026763243600726128
Before: {'शांति': 0.09575996547937393, 'शान्ति': 0.10455728322267532}
Printing dict: {'शान्ति': 0.10455728322267532, 'शांति': 0.09575996547937393}
शान्ति 0.09575996547937393
0.09575996547937393
True
शान्ति 0.07162833958864212
0.09575996547937393
शान्ति 0.026763243600726128
0.09575996547937393
['शान्ति', 'शांति']
lst: ['शान्ति']


>0.9-> 1.0 हम ताश खेल रहे हैं


>0.9-> 1.0 हम ताश खेल रहे हैं
-> ['ताश', 'खेल']
ताश 	 खेल 	: 0.01976960524916649
खेल 	 ताश 	: 0.01976960524916649
avg :  0.01976960524916649

-> ['ताश', 'खेल']
ताश 	 खेल 	: 0.01976960524916649
खेल 	 ताश 	: 0.01976960524916649
avg :  0.01976960524916649

